## Optunaで学ぶベイズハイパーパラメータチューニング超入門 – 第5回: 複数の目的変数を持つチューニング –

url: https://www.salesanalytics.co.jp/datascience/datascience195/

    ハイパーパラメータチューニングは、機械学習モデルの性能を最大化するための重要なステップです。

    前回は、「チューニング時間の短縮に貢献するプルーニング」というお話しをしました。

    url: https://www.salesanalytics.co.jp/datascience/datascience194/

    ハイパーパラメータチューニングを行う際、一般的には一つの目的変数を最適化します。
    例えば、機械学習モデルの訓練時には、精度を最大化するか、損失を最小化することが目標となることが多いです。

    しかし、実際の問題設定では、複数の目的が重要な場面もあります。例えば、精度を最大化しつつ、モデルの計算量や推論速度も考慮したい場合などが挙げられます。

    今回は、複数の目的変数を持つチューニングについてお話しします。

    要は、多目的ベイズ最適化です。

## マルチオブジェクティブチューニングとは？
    マルチオブジェクティブチューニングは、複数の目的変数を同時に最適化するチューニング手法を指します。
    これにより、トレードオフの関係にある複数の目的を考慮しつつ、最適なハイパーパラメータを探索することができます。

    Optunaは、マルチオブジェクティブチューニングをサポートしています。
    基本的な使用方法は、単一目的のチューニングと似ていますが、目的関数が複数返り値を持つ点が異なります。
    Optunaは、この複数の返り値を元にParetoフロントを計算し、最適なハイパーパラメータの組み合わせを探索します。

    具体的な実装には、create_studyメソッドでdirections引数を用いて、各目的の最大化・最小化の方向を指定します。そして、目的関数は複数のスカラー値をリストとして返すように設計します。

## コード例
    目的変数が1つの例と、2つの例を示します。

## 目的変数が1つの例
    x**2 + y**2が最小になるx∈[−10,10], y∈[−10,10]を求めます。
    以下、コードです。

In [2]:
pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 12.0 MB/s eta 0:00:00


In [3]:
import optuna

# 目的関数の定義
def objective(trial):
    # ハイパーパラメータのサンプリング
    x = trial.suggest_float("x", -10, 10)
    y = trial.suggest_float("y", -10, 10)

    # 目的関数の計算
    obj1 = x**2 + y**2

    return obj1

# スタディの作成
study = optuna.create_study(direction='minimize')

# 最適化の実行
study.optimize(objective, n_trials=100)

# 結果の確認
print(study.best_value)
print(study.best_params)

[I 2024-01-28 01:12:19,214] A new study created in memory with name: no-name-683807c3-6d6d-47f0-b8d5-0818b035f3a7
[I 2024-01-28 01:12:19,218] Trial 0 finished with value: 109.72712310380736 and parameters: {'x': 5.948002914547336, 'y': -8.622550923731547}. Best is trial 0 with value: 109.72712310380736.
[I 2024-01-28 01:12:19,223] Trial 1 finished with value: 87.0548206504757 and parameters: {'x': -8.818044015563359, 'y': 3.049085172648173}. Best is trial 1 with value: 87.0548206504757.
[I 2024-01-28 01:12:19,227] Trial 2 finished with value: 106.61076042737756 and parameters: {'x': 4.309845490869115, 'y': -9.38274971808439}. Best is trial 1 with value: 87.0548206504757.
[I 2024-01-28 01:12:19,231] Trial 3 finished with value: 165.0819509489392 and parameters: {'x': 8.11108497674929, 'y': -9.964549736385042}. Best is trial 1 with value: 87.0548206504757.
[I 2024-01-28 01:12:19,234] Trial 4 finished with value: 45.94533264801565 and parameters: {'x': -5.237939055370986, 'y': 4.302246750

0.022118168822573375
{'x': 0.08289921205098283, 'y': -0.1234742461564336}


## 目的変数が2つの例
    目的変数が1つの例に対し、目的変数を1つ追加し、目的変数を2つにします。ちなみに、追加する目的変数は(x−5)**2+(y−5)**2です。

    そのことで、目的変数を2つ持つになります。

    ・1つ目の目的変数：原点からの距離を最小化
    ・2つ目の目的変数：点(5, 5)からの距離を最小化

In [4]:
import optuna
from optuna.multi_objective import create_study
import math

# 目的関数の定義
def objective(trial):
    # ハイパーパラメータのサンプリング
    x = trial.suggest_float("x", -10, 10)
    y = trial.suggest_float("y", -10, 10)

    # 2つの目的関数の計算
    obj1 = x**2 + y**2
    obj2 = (x - 5)**2 + (y - 5)**2

    return obj1, obj2

# マルチオブジェクティブのスタディを作成（2つの目的が最小化の場合）
study = create_study(["minimize", "minimize"])

# スタディの最適化
study.optimize(objective, n_trials=100)

# マルチオブジェクティブの結果（Paretoフロント）を表示
pareto_front_trials = study.get_pareto_front_trials()
for trial in pareto_front_trials:
    print("Params {}: Values = {}".format(trial.params, trial.values))

<ipython-input-4-2b7ff1efb6c0>:18: FutureWarning: create_study has been deprecated in v2.4.0. This feature will be removed in v4.0.0. See https://github.com/optuna/optuna/releases/tag/v2.4.0.
  study = create_study(["minimize", "minimize"])
/usr/local/lib/python3.10/dist-packages/optuna/multi_objective/study.py:116: FutureWarning: NSGAIIMultiObjectiveSampler has been deprecated in v2.4.0. This feature will be removed in v4.0.0. See https://github.com/optuna/optuna/releases/tag/v2.4.0.
  mo_sampler = sampler or multi_objective.samplers.NSGAIIMultiObjectiveSampler()
/usr/local/lib/python3.10/dist-packages/optuna/multi_objective/samplers/_nsga2.py:91: FutureWarning: RandomMultiObjectiveSampler has been deprecated in v2.4.0. This feature will be removed in v4.0.0. See https://github.com/optuna/optuna/releases/tag/v2.4.0.
  self._random_sampler = multi_objective.samplers.RandomMultiObjectiveSampler(seed=seed)
[I 2024-01-28 01:13:18,276] A new study created in memory with name: no-name-aeee2

Params {'x': 2.6034467686976797, 'y': 5.735346980908975}: Values = (39.67214006886408, 6.284202572797538)
Params {'x': -0.6785149116527158, 'y': 3.3007150806947383}: Values = (11.355102529260765, 35.13310083884054)
Params {'x': 1.8672070781147951, 'y': 5.474464984277745}: Values = (33.45622913664512, 10.039508512719719)
Params {'x': 5.840319789232929, 'y': 2.304973431878283}: Values = (39.42223776217051, 7.969305551058394)
Params {'x': 3.747019830609082, 'y': -0.012809325585216413}: Values = (14.04032168979966, 26.698216639561007)
Params {'x': 2.109361318030391, 'y': -1.2504667852704117}: Values = (6.013072351067427, 47.42412702346763)
Params {'x': 4.600344235773617, 'y': 5.016338537202561}: Values = (46.326819407439075, 0.15999167767729347)
Params {'x': 2.0802564139388107, 'y': 3.2814085025868085}: Values = (15.095108508582562, 11.478459343326369)
Params {'x': 5.022038938288107, 'y': 3.8879377936519077}: Values = (40.3369353849888, 1.2371680655886539)


/usr/local/lib/python3.10/dist-packages/optuna/multi_objective/study.py:448: FutureWarning: FrozenMultiObjectiveTrial has been deprecated in v2.4.0. This feature will be removed in v4.0.0. See https://github.com/optuna/optuna/releases/tag/v2.4.0.
  multi_objective.trial.FrozenMultiObjectiveTrial(self.n_objectives, t)


    以下の2つの目的変数を同時に最小化するx∈[−10,10], y∈[−10,10]は存在しません。

    ・1つ目の目的変数：原点からの距離を最小化
    ・2つ目の目的変数：点(5, 5)からの距離を最小化

    そのため、Paretoフロントに属する解とその目的変数の値を出力します。

## Paretoフロントとは？
    Paretoフロント（パレート解）は多目的（マルチオブジェクティブ）最適化の文脈で頻繁に使われる最適解の集合の概念です。

    多目的最適化では、複数の目的関数を同時に最適化することを目指しますが、
    これらの目的関数は通常、互いにトレードオフの関係にあります。

    つまり、一つの目的変数を改善することで、他の目的変数が悪化する可能性があります。

    言い換えれば、Paretoフロントの解をさらに改善するには、少なくとも1つの他の目的変数を犠牲にしなければなりません。

    例えば、車の設計における「燃費」と「加速性能」を考えると、これらは互いにトレードオフの関係にあります。

    燃費を向上させるためには、車の重量を減らす、エンジンの出力を抑えるなどの対策が考えられますが、これにより加速性能が悪化する可能性があります。
    逆に、加速性能を向上させるためには、より大きなエンジンやターボを搭載するなどの対策が考えられますが、燃費が悪化する可能性があります。

    このようなトレードオフの関係を持つ目的間での最適なバランスを見つけるために、Paretoフロントが使用されます。

## 解を1つに絞るにはどうすればいいのか？
    多目的（マルチオブジェクティブ）最適化には、Paretoフロントと呼ばれる複数の解が存在します。困ったことにこれらの解は、トレードオフの関係にあります。

    では、1つの解に絞るにはどうすればいいでしょうか？

    最適な解を1つだけ選ぶためには、例えば以下のようなアプローチが考えられます。

    ・ビジネス要件やドメイン知識を考慮: ある目的変数が他の目的変数よりも重要であると判断される場合、その目的変数を重視して解を選択します。
    ・重み付き和を使用: 各目的変数に重みを割り当て、重み付き和を計算します。この重み付き和が最小（または最大）となる解を選択します。
    ・意思決定者との対話: エンドユーザーやステークホルダーとの対話を通じて、どの解が最も実用的かを判断します。


    以下は、重み付き和を使用して最適な解を1つ選択する例です。

In [5]:
import optuna
from optuna.multi_objective import create_study
import math

# 目的関数の定義
def objective(trial):
    # ハイパーパラメータのサンプリング
    x = trial.suggest_float("x", -10, 10)
    y = trial.suggest_float("y", -10, 10)

    # 2つの目的関数の計算
    obj1 = x**2 + y**2
    obj2 = (x - 5)**2 + (y - 5)**2

    return obj1, obj2

# マルチオブジェクティブのスタディを作成（2つの目的が最小化の場合）
study = create_study(["minimize", "minimize"])

# スタディの最適化
study.optimize(objective, n_trials=100)

# マルチオブジェクティブの結果（Paretoフロント）を取得
pareto_front_trials = study.get_pareto_front_trials()

# 重み付き和を使用して最適なトライアルを選択
weights = [0.5, 0.5]
best_trial = min(pareto_front_trials,
    key=lambda t: sum(w*v for w, v in zip(weights, t.values)))

print("Best trial by weighted sum:")
print("  Params: {}".format(best_trial.params))
print("  Values: {}".format(best_trial.values))

<ipython-input-5-7a8765dab4ac>:18: FutureWarning: create_study has been deprecated in v2.4.0. This feature will be removed in v4.0.0. See https://github.com/optuna/optuna/releases/tag/v2.4.0.
  study = create_study(["minimize", "minimize"])
[I 2024-01-28 01:15:19,050] A new study created in memory with name: no-name-937fd934-3df1-4151-9e96-61d08d2c6c09
[I 2024-01-28 01:15:19,054] Trial 0 finished with values: (148.33545157507007,) with parameters: {'x': -9.94322405186008, 'y': 7.033331147442247}.
[I 2024-01-28 01:15:19,137] Trial 1 finished with values: (45.8494355260799,) with parameters: {'x': 0.1002355672715396, 'y': 6.770479182239146}.
[I 2024-01-28 01:15:19,211] Trial 2 finished with values: (19.098146131369727,) with parameters: {'x': 4.27902647751244, 'y': 0.8877378757928511}.
[I 2024-01-28 01:15:19,286] Trial 3 finished with values: (11.080355995440724,) with parameters: {'x': -1.9679553596700234, 'y': 2.6846801853827493}.
[I 2024-01-28 01:15:19,359] Trial 4 finished with value

Best trial by weighted sum:
  Params: {'x': 3.189816709010085, 'y': 1.306000464397366}
  Values: (11.880567850086063, 16.922396116011555)


    このコードでは、重みweightsを[0.5, 0.5]としていますが、この重みは目的に応じて調整することができます。

## まとめ
    今回は、複数の目的変数を持つチューニングについてお話ししました。

    目的変数を複数にすると、Paretoフロントという最適解の集合が登場します。複数の目的変数を同時に最適化する解が、通常は存在しないためです。

    多くの場合、トレードオフの関係が生まれます。つまり、一つの目的を改善することで、他の目的が悪化する可能性があります。

    実務では1つの解に絞る必要もあることでしょう。

    ・ビジネス要件やドメイン知識を考慮: ある目的変数が他の目的変数よりも重要であると判断される場合、その目的変数を重視して解を選択します。
    ・重み付き和を使用: 各目的変数に重みを割り当て、重み付き和を計算します。この重み付き和が最小（または最大）となる解を選択します。
    ・意思決定者との対話: エンドユーザーやステークホルダーとの対話を通じて、どの解が最も実用的かを判断します。
    
    ちなみに、この多目的ベイズ最適化は、厳密な多目的最適化ではありませんので、その点だけ注意しましょう。

    次回は、scikit-learnとOptunaを統合したOptunaSearchCVを中心にお話しします。